In [1]:
import pandas as pd
import json
import numpy as np
import requests

In [2]:
# load feather files
test_data = 'test_FD001.feather'
rul_data = 'rul_FD001.feather'
test_data_file_path = f'../data/processed/{test_data}'
rul_data_file_path = f'../data/processed/{rul_data}'

test_df = pd.read_feather(test_data_file_path)
rul_df = pd.read_feather(rul_data_file_path)
test_df.head()

,unit_number,time,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [3]:
rul_df.head()

,rul
0,112
1,98
2,69
3,82
4,91


In [4]:
test_df.shape

(13096, 26)

In [5]:
rul_df.shape

(100, 1)

In [6]:
test_df['unit_number'].nunique()

100

In [7]:
units = test_df['unit_number'].unique()
units

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [8]:
unit = np.random.choice(units, size=1)[0]
empirical_rul = rul_df.iloc[unit].values[0]

In [9]:
rul_df[70:]

,rul
70,118
71,50
72,131
73,126
74,113
75,10
76,34
77,107
78,63
79,90


In [10]:
test_df.head()

,unit_number,time,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [11]:
unit

81

In [12]:
port = 80
host = 'host.docker.internal' 
url = f'http://{host}:{port}/stream_predict'

In [13]:
for index, row in test_df[test_df['unit_number'] == unit].iterrows():
    data_len = test_df.loc[unit].shape[0] + 1
    percent_complete = index * 1.0 / data_len
    new_measurement = row.to_dict()
    response = requests.post(url, json=new_measurement)
    print(response.json()['prediction'])

[145.39146914]
[158.08954335]
[156.95679588]
[135.5342782]
[154.30958808]
[155.18780683]
[148.37271717]
[159.35302328]
[156.44021698]
[143.95025477]
[147.29962309]
[156.20388395]
[150.9676978]
[168.89356187]
[146.1707848]
[147.64406404]
[167.63252539]
[150.02224331]
[146.66467054]
[149.30977271]
[155.07583878]
[148.4439051]
[161.5187866]
[158.76595041]
[158.66533054]
[147.94461998]
[149.15025302]
[149.51840627]
[161.92630854]
[146.84611203]
[144.80844917]
[158.95764404]
[166.11064089]
[134.26944805]
[133.6901158]
[137.05803892]
[158.85611514]
[165.47000977]
[159.65122531]
[119.78653183]
[149.2679812]
[150.43909759]
[148.74143383]
[141.78973879]
[141.3470043]
[143.95481562]
[141.84199017]
[144.72780098]
[137.5987043]
[157.27534365]
[158.31194893]
[145.99358306]
[153.21197882]
[134.81021027]
[128.84325374]
[161.08800163]
[153.10673637]
[155.74087074]
[137.39542308]
[141.98965134]
[139.33723579]
[133.40828099]
[143.04262928]
[141.43940596]
[161.90198298]
[150.32837008]
[153.81207577]
[139

In [14]:
empirical_rul

9